In [1]:
# !pip install pandas
# !pip install scikit-learn

In [2]:
# imports:
import os
import numpy as np
import pandas as pd
from PIL import Image

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image

from sklearn.model_selection import train_test_split

2023-04-19 20:15:24.362051: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 20:15:24.466689: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 20:15:24.467771: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 20:15:25.472495: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# selecting paths and image size:
IMG_SIZE = 150
ROCK_DIR = './/images//rock'
PAPER_DIR = './/images//paper'
SCISSORS_DIR = './/images//scissors'

In [4]:
# loading datasets:
rock_files = os.listdir(ROCK_DIR)
rock_df = pd.DataFrame({'file_path': [os.path.join(ROCK_DIR, f) for f in rock_files], 'rock': 1, 'paper': 0, 'scissors': 0})
paper_files = os.listdir(PAPER_DIR)
paper_df = pd.DataFrame({'file_path': [os.path.join(PAPER_DIR, f) for f in paper_files], 'rock': 0, 'paper': 1, 'scissors': 0})
scissors_files = os.listdir(SCISSORS_DIR)
scissors_df = pd.DataFrame({'file_path': [os.path.join(SCISSORS_DIR, f) for f in scissors_files], 'rock': 0, 'paper': 0, 'scissors': 1})
df = pd.concat([rock_df, paper_df, scissors_df], ignore_index=True)

In [5]:
# read and preprocess data from the images:
def read_image(file_path):
    img = load_img(file_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img)
    img_array /= 255.0
    return img_array

X = np.zeros((len(df), IMG_SIZE, IMG_SIZE, 3))
y = df[['rock', 'paper', 'scissors']].values

for i, file_path in enumerate(df['file_path']):
    img = read_image(file_path)
    X[i] = img

In [6]:
# split trains and tests:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
# prepare the convolutional neural network:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# evaluate the model on the testing data
test_loss, test_acc = model.evaluate(X_test, y_test)

Epoch 1/5
18/18 [==============================] - 15s 794ms/step - loss: 0.7432 - accuracy: 0.6702 - val_loss: 0.3194 - val_accuracy: 0.8881
Epoch 2/5
18/18 [==============================] - 16s 895ms/step - loss: 0.1772 - accuracy: 0.9456 - val_loss: 0.1076 - val_accuracy: 0.9650
Epoch 3/5
18/18 [==============================] - 17s 938ms/step - loss: 0.0747 - accuracy: 0.9719 - val_loss: 0.0271 - val_accuracy: 0.9930
Epoch 4/5
18/18 [==============================] - 17s 947ms/step - loss: 0.0075 - accuracy: 0.9982 - val_loss: 0.0153 - val_accuracy: 0.9930
Epoch 5/5
5/5 [==============================] - 1s 218ms/step - loss: 0.0173 - accuracy: 0.9860


In [8]:
# print accuracy:
print('Test accuracy:', test_acc)

Test accuracy: 0.9860140085220337


In [9]:
# save model:
model.save('my_model.h5')

In [10]:
# test real model:
model = keras.models.load_model('my_model.h5')

img = Image.open("test.jpg")
img = img.resize((150, 150))
img = img.convert("RGB")

img_array = np.array(img)
img_array = np.expand_dims(img_array, axis=0)
prediction = model.predict(img_array)

# Preprocess the image data
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.0

# Make a prediction on the image
prediction = model.predict(x)

# Print the prediction result
classes = ['paper', 'rock', 'scissors']
predicted_class = classes[np.argmax(prediction)]
print(f'Predicted class: {predicted_class}')

1/1 [==============================] - 0s 78ms/step
Predicted class: rock
